In [2]:
from pymongo import MongoClient
client = MongoClient()   #Connect to MongoDB client

In [5]:
db = client.mapreduce #database initializing
#insert observations to database
insert_observations = db.observations.insert_many([
    {
    #observationTimestamp: Date.parse("Mon, 25 Dec 1995 12:34:56 GMT"),
    "month" : 12,
    "year" : 1995,
    "Time" : "12:34:56 GMT",
     "Date" : 25,
    "family":     "Sharks",
    "species":    "Carcharodon carcharias",
    "numAnimals": 3
    },
    {
    #observationTimestamp: Date.parse("Tue, 12 Dec 1995 16:17:18 GMT"),
    "month" : 12,
    "year" : 1995,
    "Time" : "16:17:18 GMT",
     "Date" : 12,
    "family":     "Sharks",
    "species":    "Carcharias taurus",
    "numAnimals": 4
    }    
]       
)    


In [7]:
insert_observations.inserted_ids #two documents/observations are inserted in to database

[ObjectId('5a5eca0bb6c97a36d44a768f'), ObjectId('5a5eca0bb6c97a36d44a7690')]

#### How many Carcharias taurus have been found?

In [26]:
count = 0
for a in db.observations.find({'species':'Carcharias taurus'}):
    count+=1
    pprint.pprint(a)
print("Number of Observations with Carcharias taurus species found are : ",count)

{'Date': 12,
 'Time': '16:17:18 GMT',
 '_id': ObjectId('5a5eca0bb6c97a36d44a7690'),
 'family': 'Sharks',
 'month': 12,
 'numAnimals': 4,
 'species': 'Carcharias taurus',
 'year': 1995}
Number of Observations with Carcharias taurus species found are :  1


### Map Reduce

In [9]:
from bson.code import Code
#defining map function using Code function in  java script syntax for MongoDB

#### Mapper

In [14]:
mapper = Code("""
              function () {
                var year = this.year;
                var month = this.month;
                emit(year + "-" + month, this.numAnimals);
               }
             """)

#### Reducer

In [15]:
reducer = Code("""
               function(key,values){
                   return Array.sum(values);
               }
               """)

In [16]:
# Execute Map-Reduce function
result = db.observations.map_reduce(mapper,reducer,'monthlySharkReport')

In [17]:
import pprint
for observation in result.find():
    pprint.pprint(observation)

{'_id': '1995-12', 'value': 7.0}
